In [1]:
from argparse import ArgumentParser, ArgumentDefaultsHelpFormatter
import pandas as pd
import numpy as np
import json
import ast
from sklearn.model_selection import train_test_split

from utils import Indexer

from tqdm import tqdm

In [2]:
links = 'data/links.csv'
metadata = 'data/movies_metadata.csv'
credit = 'data/credits.csv'
rating = 'data/ratings.csv'

train_dir = 'processed_data/rating_train.csv'
test_dir = 'processed_data/rating_test.csv'

overview = 'processed_data/overviews.csv'



In [12]:
data = pd.read_csv(metadata)
data.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [164]:
# Infer a vector for a new doc
test = ["Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences"]
texts = [[word for word in document.lower().split() if word not in stoplist]
         for document in test]
print(f"test sample: {texts}")
vector = model.infer_vector(texts[0])
print(f"vector: {vector}")

test sample: [['led', 'by', 'woody,', "andy's", 'toys', 'live', 'happily', 'his', 'room', 'until', "andy's", 'birthday', 'brings', 'buzz', 'lightyear', 'onto', 'scene.', 'afraid', 'losing', 'his', 'place', "andy's", 'heart,', 'woody', 'plots', 'against', 'buzz.', 'but', 'when', 'circumstances', 'separate', 'buzz', 'woody', 'from', 'their', 'owner,', 'duo', 'eventually', 'learns', 'put', 'aside', 'their', 'differences']]


In [162]:
overview_sentences = data['overview'].astype(str)
mId = data['mId'].astype(int)
print("Number of sentence: %d"%(len(overview_sentences)) )
type(overview_sentence)
overview_sentence[:2]

Number of sentence: 45463


["Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences.",
 "When siblings Judy and Peter discover an enchanted board game that opens the door to a magical world, they unwittingly invite Alan -- an adult who's been trapped inside the game for 26 years -- into their living room. Alan's only hope for freedom is to finish the game, which proves risky as all three find themselves running from giant rhinoceroses, evil monkeys and other terrifying creatures."]

In [163]:
import pprint
# Some preprocessing
stoplist = set('for a of the and to in'.split(' '))
texts = [[word for word in document.lower().split() if word not in stoplist]
         for document in overview_sentences]
print(texts[:1])

# Count word frequencies
from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

# Only keep words that appear more than once
processed_corpus = [[token for token in text if frequency[token] > 1] for text in texts]
# pprint.pprint(processed_corpus)

from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
documents = [TaggedDocument(texts[i], [mId[i]]) for i in range(len(texts))] # You need the [] for mId[i], because both input need to be iterable, e.g., str, list, dict
print(type(documents))
print(f"Shape of document: {len(documents)}")
print(f"Sample of documents: {documents[:2]}")
model = Doc2Vec(iter(documents), vector_size=5, window=2, min_count=1, workers=4)


[['led', 'by', 'woody,', "andy's", 'toys', 'live', 'happily', 'his', 'room', 'until', "andy's", 'birthday', 'brings', 'buzz', 'lightyear', 'onto', 'scene.', 'afraid', 'losing', 'his', 'place', "andy's", 'heart,', 'woody', 'plots', 'against', 'buzz.', 'but', 'when', 'circumstances', 'separate', 'buzz', 'woody', 'from', 'their', 'owner,', 'duo', 'eventually', 'learns', 'put', 'aside', 'their', 'differences.']]


ModuleNotFoundError: No module named 'gensim'

In [ ]:
# # Parsing the overview sentences
overview_sentences = []
for overview_sentences, mId in zip(overview_sentences, mId):
    print(overview_sentences)
    # overview_sentences.append(item)


In [ ]:
# Tokenize the sentences
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

vocab_size = 10000 # Size of word corpus
embedding_dim = 16 # dimension of output vector
max_length = 32 # Maximum length of token
trunc_type = 'post'
padding_type = 'post'
oov_tok = '<ooV>'
data_size = data.shape[0] # Size of training dataset

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(overview_sentences)
word_index = tokenizer.word_index
# print(word_index)
overview_sequences = tokenizer.texts_to_sequences(overview_sentences)
overview_padded = pad_sequences(overview_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
# print("\nWord Index = " , word_index)
# print(f"Sequences[:3]: {overview_sentence[:3]}")
print("Padded Sequences:")
print(overview_padded[:, :3])
print(overview_padded.shape)


In [ ]:
overview_sequences1 = tokenizer.texts_to_matrix(overview_sentences, mode='binary')

In [ ]:
print("Padded Sequences:")
print(overview_sequences1[:, :3])
print(overview_sequences1.shape)

In [ ]:
import 


In [ ]:
x1 = np.arange(10)
x2 = x1*2
y = x1*2
y

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
x1 = np.arange(10)
x2 = x1*2
y = x1*2

plt.figure(figsize=(12.2, 4.5)) # width=12.2in, height=4.5in
plt.scatter(x1, y, color='green', label='Line1')
plt.scatter(x2, y, color='red', label='Line2')
plt.plot(x1, y, alpha=0.35) #	Plot y versus x as lines and/or markers.
plt.xticks(rotation=45)
plt.title(label='Title of this figure')
plt.xlabel('X label', fontsize=18)
plt.ylabel('Y label', fontsize=18)
plt.legend(loc='best')
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
data = [[30, 25, 50, 20],
[40, 23, 51, 17],
[35, 22, 45, 19]]
X = ['2011', '2012', '2013', '2014']
fig = plt.figure()
# ax = fig.add_axes([0,0,1,1])
ax.bar(X, data[0], color = 'b', width = 0.25)
ax.bar(X, data[1], color = 'g', width = 0.25)
ax.bar(X, data[2], color = 'r', width = 0.25)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# df = pd.DataFrame(np.array([[94.13, 94.01], [44.91, 43.97]]),
#     index = ["Fashion MNIST", "Cifar100"],
#     columns=pd.Index(['GPU',' CPU'], name='Device')
# )
df = pd.DataFrame({'GPU': [94.01, 43.97],
        'CPU': [94.01, 44.91]},
        index=["Fashion MNIST", "Cifar100"]
)

df.plot.bar(xlabel='dataset', ylabel='Acc(%)', title='DNN Model1: Flatten()-->Dense(128)-->Dense(10)', rot=30)

# acc = [[94.13, 94.01], [44.91, 43.97]] # CPU acc and GPU acc for Fashion MNIST and Cifar100
# time = [[2.812, 2.871], [1.1, 2.2]]
# X = np.arange(4)
# fig = plt.figure()
# ax = fig.add_axes([0,0,1,1])
# ax.bar(X + 0.00, data[0], color = 'b', width = 0.25)
# ax.bar(X + 0.25, data[1], color = 'g', width = 0.25)

In [ ]:
df = pd.DataFrame({'GPU': [95.5, 89.06],
        'CPU': [96.77, 88.61]},
        index=["Fashion MNIST", "Cifar100"]
)

df.plot.bar(xlabel='dataset', ylabel='Acc(%)', title='DNN Model2: Con2D(32)-->Conv2D(64)-->Flatten()-->Dense(128)-->Dense(10)', rot=30)


In [ ]:
df = pd.DataFrame({'GPU': [2.871, 2.2],
        'CPU': [2.812, 1.1]},
        index=["Fashion MNIST", "Cifar100"]
)

df.plot.bar(xlabel='dataset', ylabel='Time(sec/epoch)', title='DNN Model1: Flatten()-->Dense(128)-->Dense(10)', rot=30)

In [ ]:
df = pd.DataFrame({'GPU': [4.2, 3.3],
        'CPU': [14.7, 11.9]},
        index=["Fashion MNIST", "Cifar100"]
)

df.plot.bar(xlabel='dataset', ylabel='Time(sec/epoch)', title='DNN Model2: Con2D(32)-->Conv2D(64)-->Flatten()-->Dense(128)-->Dense(10)', rot=30)


In [ ]:
df = pd.DataFrame({'GPU': [3022, 3429],
        'CPU': [23679, 9526]},
        index=["VGG19", "ResNet50"]
)

df.plot.bar(xlabel='DNN Model', ylabel='Time(sec/epoch)', title='What is the Best DNN model for IMDB-Wiki  Dataset(on similar ACC)', rot=30)


In [ ]:
df = pd.DataFrame({'Time(sec/epoch)': [184, 99, 72, 78, 67],
        'Throughput(sec/step)': [2, 0.988, 0.719, 0.779, 0.668]},
        index=[2, 4, 8, 16, 28]
)

df.plot.bar(xlabel='Cores', ylabel='Time(sec)', title='ResNet50 on IMDB-Wiki Dataest(on CPU and batch_size=32)', rot=30, subplots=True)


In [ ]:
df = pd.DataFrame({'Time(sec/epoch)': [8, 10, 37, 72, 74, 361],
        # 'Throughput(step/ms)': [84, 95, 374, 715, 737, 4000]},
        'Throughput(sec/step)': [0.084, 0.095, 0.374, 0.715, 0.737, 4]},
        index=[2, 4, 8, 16, 32, 64]
)

df.plot.bar(xlabel='Batch size', ylabel='Time(sec)', title='ResNet50 on IMDB-Wiki Dataest(on GPU and cores=28)', rot=30, subplots=True)
